# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [47]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import json
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [48]:
the_cities = pd.read_csv("list_of_every_city.csv", encoding="utf-8")
the_cities.head()

,Unnamed: 0,lat,lon,temp,humidity,cloudiness,wind,temp_max
0,0,54.58,16.86,275.27,85,100,12.49,275.27
1,1,37.52,42.46,281.13,45,35,2.33,281.13
2,2,-33.28,-70.90,288.15,77,6,2.10,288.15
3,3,71.92,114.08,249.48,87,59,1.56,249.48
4,4,47.47,84.87,273.00,69,79,6.43,273.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [49]:
humidity = the_cities["humidity"].astype(float)
maxhumidity = humidity.max()
locations = the_cities[["lat", "lon"]]

In [50]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [51]:
narrowed_city_df = the_cities.loc[(the_cities["temp_max"] > 294) & (the_cities["temp_max"] < 300) & (the_cities["cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

#NOTE TEMP IS IN KELVIN, SO THE 70-80 FARENHEIGHT RANGE IS CONVERTED TO 294-300 

,Unnamed: 0,lat,lon,temp,humidity,cloudiness,wind,temp_max
0,31,15.74,-96.47,297.59,84,0,3.81,297.59
1,91,22.53,-80.91,295.15,83,0,2.10,295.15
2,160,-0.48,15.90,296.80,80,0,0.83,296.80


In [56]:
hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['lat']
    lng = narrowed_city_df.loc[i]['lon']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,Unnamed: 0,lat,lon,temp,humidity,cloudiness,wind,temp_max,Hotel Name
0,31,15.74,-96.47,297.59,84,0,3.81,297.59,Pochutla District
1,91,22.53,-80.91,295.15,83,0,2.10,295.15,Jesus Rabi
2,160,-0.48,15.90,296.80,80,0,0.83,296.80,Owando
